In [1]:
import pandas as pd # Manejo de bases de datos
import geopandas as gpd # Manejo de bases de datos geográficas
import numpy as np # Funciones numéricas
import matplotlib.pyplot as plt # Gráficas
import seaborn as sns # Gráficas
import datetime as dt
import folium
import unicodedata
import geojson
import urllib, json
import base64

from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn import neighbors
from sklearn.neural_network import MLPClassifier

from shapely.geometry import LineString, Point
from geopandas.tools import sjoin
from branca.colormap import linear

import dash
from dash import dcc
from dash import html
import dash_bootstrap_components as dbc
from dash.dependencies import Input, Output
import visdcc
from waitress import serve

### Prepare Data needed to create DashBoard

In [2]:
####################################################################################################
############## Read Bogota spatial characteristics previously prepared and organized.###############
####################################################################################################
#path = '/home/ubuntu/javeriana/MOTUS-PUJ/Bog_Estimation/reg_ml/partial_spatial_block.pkl'
path = '/home/ubuntu/javeriana/MOTUS-PUJ/Step_2/2_ML_Preps/Outputs/partial_spatial_block.pkl'
spatial_block = pd.read_pickle(path, compression='gzip')
spatial_block = gpd.GeoDataFrame(spatial_block)
spatial_block.crs = 'EPSG:4326'
spatial_block.to_crs(epsg=4326, inplace=True)
#Do not consider city blocks with area greater than 400 m^2
spatial_block = spatial_block[ spatial_block['Area_km2'] < 0.4 ]
Loc_code = list(range(1,20)) #Create localities Id list

###################################################################################################
############################Load RT estimated for each locality####################################
###################################################################################################
# Reading localities R_t
#base_path = '/home/ubuntu/javeriana/MOTUS-PUJ/Bog_Estimation/ECM/R_loc_loc/'
base_path = '/home/ubuntu/javeriana/MOTUS-PUJ/Step_1/RT_outputs/'

loc_R_list = ['usaquen.pkl', 'chapinero.pkl', 'santafe.pkl', 'sancristobal.pkl', 'usme.pkl',
             'tunjuelito.pkl', 'bosa.pkl', 'kennedy.pkl', 'fontibon.pkl', 'engativa.pkl',
             'suba.pkl', 'barriosunidos.pkl', 'teusaquillo.pkl', 'losmartires.pkl', 'antonionariño.pkl',
             'puentearanda.pkl', 'lacandelaria.pkl', 'rafaeluribeuribe.pkl', 'ciudadbolivar.pkl']

R_list = []

for i in range(len(loc_R_list)):
    path_file = base_path+loc_R_list[i]
    R_list.append(pd.read_pickle(path_file))
    
for i in range(len(loc_R_list)):
    R_list[i].reset_index(drop=False, inplace=True)
    
R_df = pd.DataFrame(index = R_list[0]['Time Stamp'])
R_df.reset_index(drop=False, inplace=True)

for i in range(len(loc_R_list)):
    R_df[loc_R_list[i]] = 0
    R_df[loc_R_list[i]] = R_list[i]['R'].tolist()
    
###################################################################################################
###########################Active cases count for each locality####################################
###################################################################################################
#fechas_df = pd.read_csv('/home/ubuntu/javeriana/MOTUS-PUJ/Bog_Estimation/ECM/fechas_sintomas.csv')
fechas_df = pd.read_csv('/home/ubuntu/javeriana/MOTUS-PUJ/Step_1/Outputs/fechas_sintomas.csv')
fechas_df = fechas_df.rename(columns={'LOCALIDAD_ASIS': 'CASOS_REPORTADOS'})#Rename Column

#Change str to datetime objects
fechas_df['FECHA_DE_INICIO_DE_SINTOMAS'] = pd.to_datetime(fechas_df['FECHA_DE_INICIO_DE_SINTOMAS'], format='%Y-%m-%d')
fechas_df = fechas_df.sort_values(by='FECHA_DE_INICIO_DE_SINTOMAS', ascending=True)
fechas_df = fechas_df.reset_index(drop=True)

#Drop 0 and 21 id codes since they don´t belong to Bogotá geography 
index_0 = fechas_df[ fechas_df['CODIGO_LOCALIDAD'] == 0 ].index
index_21 = fechas_df[ fechas_df['CODIGO_LOCALIDAD'] == 21 ].index

fechas_df.drop(index_0, inplace = True)#drop fuera de bogotá and sin dato rows since we cannot calculate infection 
fechas_df.drop(index_21, inplace = True)# density for them (we don´t know their reference population)

fechas_df = fechas_df.reset_index(drop = True)

com_index = len(fechas_df['FECHA_DE_INICIO_DE_SINTOMAS']) #column length 
start_date = fechas_df.loc[0, 'FECHA_DE_INICIO_DE_SINTOMAS'] #start of the pandemic in Bogotá 
end_date = fechas_df.loc[com_index-1, 'FECHA_DE_INICIO_DE_SINTOMAS'] #Last reported date 

pan_days = end_date - start_date
pan_days = int(pan_days.days) #Days passed since pandemic start to last reported date grid x axis 
local_bog = 20 # number of rows y axis we are not considering 0 and 21 codes 

#create grid and fill it with reported cases y axis correspond to Bogota localities id code, x axis correspond to 
#number of days passed since pandemic started that way 0 index -> 2020-02-06, 1->2020-02-07 and so on

grid = np.ndarray([local_bog, pan_days+1]) #create grid
grid.fill(0) #fill grid with 0 

#Method used to fill grid with reported cases 
def fillGrid(date, code, cases):
    col = date - start_date
    col = int(col.days)
    grid[code-1][col] = cases
    

#Fill grid with cases 
fechas_df.apply(lambda row: fillGrid(row['FECHA_DE_INICIO_DE_SINTOMAS'], row['CODIGO_LOCALIDAD'], int(row['CASOS_REPORTADOS']) ), axis=1)
#function to count active cases in determined date
def ActiveCases (date, code):
    col = date - start_date
    col = int(col.days)
    if col >= 15:
        Active = sum(grid[code-1][col-15:col+1])
    else: 
        Active = sum(grid[code-1][0:col+1])
    return int(Active)

### Functions

In [3]:
#Function used to estimate RT for each city block
def singularizeRT(estrato, popden, hosp, ips, itur, sitp, comer, trips, cases, RT):
    a_rt = RT-estrato+popden-hosp-ips+itur+sitp+comer+trips+cases
    return a_rt

#Function used to estimate cases for each city block 
def single_cases(ppl_block, caseLoc, PobLoc, ID):
    x = (100*caseLoc)/(PobLoc)
    y = (x*ppl_block)/100
    return y


#Prepare a DataFrame to perform classification
def ClassPrepDF(selec_date, selec_str, spatial_block):
    #Add locality Active cases in interest date
    Active_cases = []
    for i in range(len(Loc_code)): #Count Cases
        Active_cases.append(ActiveCases(selec_date, Loc_code[i]))
    # Add Cases 
    spatial_block_date = spatial_block.copy()
    spatial_block_date['cases_'+selec_str] = spatial_block_date.apply(lambda row: Active_cases[row['LOCid']-1], axis=1)
    #Estimate Number of active cases for each city block
    spatial_block_date['block_cases_'+selec_str] = spatial_block_date.apply(lambda row: single_cases(row['ppl_block'], 
                                                                                                          row['cases_'+selec_str], 
                                                                                                          row['PopLoc'], 
                                                                                                          row['LOCid']), axis=1)
    
    #Add RT at locality level for specific date
    rt_list = R_df[ R_df['Time Stamp'] == selec_date ]
    rt_list = rt_list.iloc[:, 1:].values.tolist()[0]
    spatial_block_date['Rt_'+selec_str] = spatial_block_date.apply(lambda row: rt_list[row['LOCid']-1], axis=1)
    #Drop cols we dont need anymore
    spatial_block_date.drop(columns={'cases_'+selec_str, 'PopLoc'}, inplace=True)
    #Calculate RT for city block
    spatial_block_date_rt = spatial_block_date.copy()
    spatial_block_date_rt['ESTRATOPre'] = spatial_block_date_rt['ESTRATOPre']/(spatial_block_date_rt['ESTRATOPre'].max()*4)
    spatial_block_date_rt['BlockPopDen'] = spatial_block_date_rt['BlockPopDen']/(spatial_block_date_rt['BlockPopDen'].max()*4)
    spatial_block_date_rt['N_Hosp'] = spatial_block_date_rt['N_Hosp']/(spatial_block_date_rt['N_Hosp'].max()*4)
    spatial_block_date_rt['N_IPS'] = spatial_block_date_rt['N_IPS']/(spatial_block_date_rt['N_IPS'].max()*4)
    spatial_block_date_rt['N_ITur']  = spatial_block_date_rt['N_ITur']/(spatial_block_date_rt['N_ITur'].max()*4)
    spatial_block_date_rt['N_SITP'] = spatial_block_date_rt['N_SITP']/(spatial_block_date_rt['N_SITP'].max()*4)
    spatial_block_date_rt['N_Ecomer'] = spatial_block_date_rt['N_Ecomer']/(spatial_block_date_rt['N_Ecomer'].max()*4)
    spatial_block_date_rt['total_trips'] = spatial_block_date_rt['total_trips']/(spatial_block_date_rt['total_trips'].max()*4)
    spatial_block_date_rt['block_cases_'+selec_str] = spatial_block_date_rt['block_cases_'+selec_str]/(spatial_block_date_rt['block_cases_'+selec_str].max()*3)
    
    spatial_block_date_rt['BlockRT_'+selec_str] = spatial_block_date_rt.apply(lambda row: singularizeRT(row['ESTRATOPre'], 
                                                                                       row['BlockPopDen'], row['N_Hosp'],
                                                                                       row['N_IPS'], row['N_ITur'],
                                                                                       row['N_SITP'], row['N_Ecomer'],
                                                                                       row['total_trips'], 
                                                                                       row['block_cases_'+selec_str],
                                                                                       row['Rt_'+selec_str]), axis=1)
    
    # Finally generate final normalized DF with an estimated rt for each city block
    spatial_block_date['BlockRT_'+selec_str] = spatial_block_date_rt['BlockRT_'+selec_str]
    spatial_block_date['ESTRATOPre'] = spatial_block_date['ESTRATOPre']/(spatial_block_date['ESTRATOPre'].max())
    spatial_block_date['BlockPopDen'] = spatial_block_date['BlockPopDen']/(spatial_block_date['BlockPopDen'].max())
    spatial_block_date['N_Hosp'] = spatial_block_date['N_Hosp']/(spatial_block_date['N_Hosp'].max())
    spatial_block_date['N_IPS'] = spatial_block_date['N_IPS']/(spatial_block_date['N_IPS'].max())
    spatial_block_date['N_ITur']  = spatial_block_date['N_ITur']/(spatial_block_date['N_ITur'].max())
    spatial_block_date['N_SITP'] = spatial_block_date['N_SITP']/(spatial_block_date['N_SITP'].max())
    spatial_block_date['N_Ecomer'] = spatial_block_date['N_Ecomer']/(spatial_block_date['N_Ecomer'].max())
    spatial_block_date['total_trips'] = spatial_block_date['total_trips']/(spatial_block_date['total_trips'].max())
    spatial_block_date['block_cases_'+selec_str] = spatial_block_date['block_cases_'+selec_str]/(spatial_block_date['block_cases_'+selec_str].max())
    spatial_block_date['BlockRT_'+selec_str] = spatial_block_date['BlockRT_'+selec_str]/(spatial_block_date['BlockRT_'+selec_str].max())
    # Drop cols we dont need anymore
    spatial_block_date.drop(columns={'Rt_'+selec_str}, inplace=True)
    
    return spatial_block_date

# Classifier Training Function 
# model -> NN for MultiLayerPerceptron Neural Netowrok, RF for RandomForest, KNN for K Nearest Neighbors 
def TrainClassificator(model):
    #TrainDF = pd.read_pickle('/home/ubuntu/javeriana/MOTUS-PUJ/Bog_Estimation/reg_ml/ClustDFs/Labeled/TrainClass.pkl')
    TrainDF = pd.read_pickle('/home/ubuntu/javeriana/MOTUS-PUJ/Step_2/2_ML_Preps/Outputs/TrainClass.pkl')
    XTrain = TrainDF.drop(columns={'labels'})
    Xmat = XTrain.to_numpy()
    Yl = TrainDF['labels'].to_numpy()
    if model == 'NN':
        NN = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(), random_state=1)
        NN.fit(Xmat, Yl)
        return NN
    
    elif model == 'KNN':
        KNN = neighbors.KNeighborsClassifier(n_neighbors=16, weights='distance', algorithm='auto')
        KNN.fit(Xmat, Yl)
        return KNN
    
    elif model == 'RF':
        RF = RandomForestClassifier(random_state=0)
        RF.fit(Xmat, Yl)
        return RF
        
    else:
        raise ValueError('model argument must be NN, KNN or RF')
        
def Calc_date(index, mode):
    date_list = ['01/05/2020' ,'01/08/2020', '01/11/2020', '10/01/2021', '10/03/2021', '01/05/2021',
                 '20/06/2021', '01/09/2021']

    date_list2 = ['01-05-2020' ,'01-08-2020', '01-11-2020', '10-01-2021', '10-03-2021', '01-05-2021',
                  '20-06-2021', '01-09-2021']
    if mode == 0:
        return date_list[index]
    elif mode == 1:
        return date_list[index]
    
def Get_Here_Route(date, ruta):
    peak_dates4 = ['2020-05-01' ,'2020-08-01', '2020-11-01', '2021-01-10', '2021-03-01', '2021-05-01',
             '2021-06-10', '2021-09-01']
    
    API_key = '1NGxYsCA_oIhkYX-tv9HgZ3XH3tEc8rGBzQt0fz-OYY'
    
    #### ------- Latitude, longitude ----- ###### 
    Inicio = str(OD[ruta][0][0])+','+str(OD[ruta][0][1])
    Final  = str(OD[ruta][1][0])+','+str(OD[ruta][1][1])
    
    Modo = 'publicTransport'
    ### ------ Analisis tiempo ---- #### 
    Tiempo = dt.datetime.strptime(peak_dates4[date]+' 06:30:00', '%Y-%m-%d %H:%M:%S')
    Tiempo=Tiempo.strftime('%Y-%m-%dT%H:%M:%S')
    Tiempo = '&departure='+str(Tiempo)
    
    ##### -------- Tipo de ruta ----- ####### 
    ## Fastest: Más rapida 
    ## shortest: Más Corta 
    ## balanced: Balanceada
    Tipo_ruta = 'balanced'
    
    ####### ------- Solicitud -------- #####
    built_url = 'https://route.ls.hereapi.com/routing/7.2/calculateroute.json?apikey='+API_key+'&waypoint0=geo!'+Inicio+'&waypoint1=geo!'+Final+'&mode='+Tipo_ruta+';'+Modo+';traffic:enabled&legattributes=li&c&returnelevation=true&metricSystem=imperial'+str(Tiempo)
    html_data = pd.read_json(built_url)
    Datos_ruta = pd.DataFrame(pd.DataFrame(pd.DataFrame(html_data.T['route'].iloc[0])['leg'].iloc[0])['link'].iloc[0])
    Datos_ruta = pd.DataFrame(pd.concat([pd.DataFrame(x)[0] for x in Datos_ruta['shape']]))[0].reset_index(drop=True)
    Latitudes = pd.DataFrame({'latitude': Datos_ruta.apply(lambda x: float(x.split(',')[0])),'longitude': Datos_ruta.apply(lambda x: float(x.split(',')[1]))})
    Ruta_here  = LineString(list(Latitudes.apply(lambda x: (x['longitude'],x['latitude']),axis=1)))
    
    return Ruta_here

def Route_risk(route, DF):
    line_gdf = gpd.GeoDataFrame(geometry=[route])
    line_gdf['geometry'] = line_gdf['geometry'].buffer(0.00009)
    line_gdf.crs = "EPSG:4326"
    line_gdf.to_crs(epsg=4326, inplace=True)    
    
    RB_join = gpd.sjoin(line_gdf, DF, op="intersects")
    
    #RB_join.crs = "EPSG:4326"
    #RB_join.to_crs(epsg=4326, inplace=True)
    
    #Route Risk
    Route_Risk = sum(RB_join['RiskStrat'].values.tolist())/len(RB_join['RiskStrat'].values.tolist())
    return ( round(Route_Risk) )
    
    
    
    
def Route_Risk_Map(date, spatial_utam, CLF):
    #Selected date to analyse routes
    selec_date = Calc_date(date, mode=0)
    selec_str =  Calc_date(date, mode=1)
    
    selec_date = dt.datetime.strptime(selec_date, "%d/%m/%Y")
    
    #Prepare DF to perform Classification
    cityBlockDF = ClassPrepDF(selec_date, selec_str, spatial_block)
    Xdf = cityBlockDF.drop(columns={'MANCODIGO', 'ppl_block', 'Area_km2', 'BlockPopDen', 'geometry',
                               'N_Hosp', 'N_IPS', 'N_ITur', 'N_SITP', 'N_Ecomer', 'UTAM', 'LOCid',
                               'ESTRATOPre', 'total_trips'})

    Xmat = Xdf.to_numpy()
    labels = CLF.predict(Xmat)
    cityBlockDF['RiskStrat'] = labels
    
    #Init folium map object for Bogotá
    Lat = 4.61
    Long = -74.082
    m = folium.Map(location=[Lat, Long], zoom_start=12, tiles='CartoDB positron')
    
    feature_0 = folium.FeatureGroup(name='Portal Suba-Marly',  overlay=True, show=True).add_to(m)
    feature_1 = folium.FeatureGroup(name='Portal Tunal-Cl 34', overlay=True, show=True).add_to(m)
    feature_2 = folium.FeatureGroup(name='20 Julio-P.Sierra',  overlay=True, show=True).add_to(m)
    feature_3 = folium.FeatureGroup(name='Cra 90-Cl 76',       overlay=True, show=False).add_to(m)
    
    feature_4 = folium.FeatureGroup(name='San Mateo-Virrey',   overlay=True, show=True).add_to(m)
    feature_5 = folium.FeatureGroup(name='Portal Usme-Virrey', overlay=True, show=False).add_to(m)
    feature_6 = folium.FeatureGroup(name='Suba Tv 91-P.Sierra',overlay=True, show=False).add_to(m)
    feature_7 = folium.FeatureGroup(name='Portal Tunal-Cl 100',overlay=True, show=False).add_to(m)
    
    feature_8 = folium.FeatureGroup(name='Portal 80-Flores',   overlay=True, show=False).add_to(m)
    
    fs = [feature_0, feature_1, feature_2, feature_3, feature_4, feature_5, feature_6, feature_7, feature_8]
    
    for i in range(len(OD)):
        #Get Route
        Ruta_here = Get_Here_Route(date, i)
        #Calculate route risk
        risk = Route_risk(route=Ruta_here, DF=cityBlockDF)
        
        if risk == 0:
            color = 'green'
        elif risk == 1:
            color = 'orange'
        elif risk == 2:
            color = 'darkred'
        
        
        ### ---------- Punto inicial --------- #####
        folium.Marker([ OD[i][0][0], OD[i][0][1] ], icon=folium.Icon(color='blue'),popup='Inicio').add_to(fs[i])
        ### ---------- Punto final --------- #####
        folium.Marker([ OD[i][1][0], OD[i][1][1] ], icon=folium.Icon(color='green'),popup='Final').add_to(fs[i])

        Ruta_geojson_here = json.loads(json.dumps(geojson.Feature(geometry=Ruta_here)) )
        folium.GeoJson(Ruta_geojson_here,
        style_function=lambda feature: {
            'fillColor': color,
            'color': color,
            'weight': 2,'dashArray': '5, 5',
            'fillOpacity':0.2,
        },highlight_function=lambda x: {'weight':10,'fillColor':color, 'color': color},tooltip='Ruta',
        name='Ruta').add_to(fs[i])
        
    folium.TileLayer('cartodbdark_matter', overlay=True, name="night mode").add_to(m)
    folium.LayerControl(collapsed=False).add_to(m)
    
    return ( m.get_root().render() )
        
    


In [4]:
# Estiaciones Top 10 Origen-Destino
# 2. Portal de Suba - Marly 
# 3. Portal del Tunal - Calle 34
# 4. Portal 20 de Julio - Pepe Sierra
# 5. Carrera 90 - Calle 76
# 6. San Mateo - Virrey
# 7. Portal de Usme - Virrey
# 8. Suba Tv.91 -  Pepe Sierra
# 9. Portal de Tunal - Calle 100
# 10.Portal de la 80 - Flores

#Origin Destiny Transmilenio stations Top 10 pairs in peak transport hour 6:30 AM
OD = [ [(4.74671771943963,  -74.09493429905602), (4.63720427691725,  -74.06683542174328)], 
        [(4.571199917723078, -74.14004021889056), (4.621485922442711, -74.0698101293249)], 
        [(4.565847903486084, -74.09769633129297), (4.698905904113961, -74.05526829102031)], 
        [(4.704596608385601, -74.10370728859698), (4.664297929286919, -74.06105964556366)], 
        [(4.58957210258105,  -74.19848506265778), (4.676115666139605, -74.0589324054742)], 
        [(4.53164552185161,  -74.11944544719375), (4.676123676932811, -74.05892436792742)], 
        [(4.739459105686381, -74.0872815378617),  (4.698902235142769, -74.05523193350736)], 
        [(4.572105421472525, -74.14028819785999), (4.689815284126171, -74.0680973627613)], 
        [(4.709660208678614, -74.11062966777239), (4.654616791511056, -74.06220949854023)] ]

# Train
CLF = TrainClassificator(model='NN')

date_list2 = ['01-05-2020' ,'01-08-2020', '01-11-2020', '10-01-2021', '10-03-2021', '01-05-2021', 
              '20-06-2021', '01-09-2021']
render_maps = []

for i in range(len(date_list2)):
    render_maps.append(Route_Risk_Map(i, spatial_block, CLF))


c:\users\acer\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:31: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
c:\users\acer\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:31: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
c:\users\acer\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:31: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
c:\users\acer\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:31: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
c:\users\acer\appdata\local\programs\python\python37\lib\site-packages\i

In [5]:
app = dash.Dash(
    external_stylesheets=[dbc.themes.CYBORG]
)

image_filename = 'Route_risk_scale.png' #image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

### --- Principal Title --- ### 
app.title = 'PUJ MOTUS TG'
### ---------------- Logo - Image ------------------ ###

Arriba = dbc.NavbarSimple(children=[
html.Img(src='https://www.javeriana.edu.co/recursosdb/20129/601896/escudoPUJ-Bogota_rgb-azul_lateral.png/3eec1aa5-947a-1655-7fe7-6c4dfb397793?t=1611842495478',
         width=230,height=100)],
    brand="MOTUS Science - PUJ",
    brand_href="#",
    color="navy",
    dark=True,
)



Estilo_arriba_caja = {'textAlign': 'center','height': 75, 'display': 'block', 
                      'margin-left': 'auto','margin-right': 'auto','width': '100%', 
                      'color': '#ffffff',"background-color": 'navy'}

Estilo_caja = {"max-width": "2000px", 'height': 'calc(130vh - 60px)', 'overflowY': 'scroll',
               "opacity": 0.9, "background-color": "black",'textAlign': 'center', 'width': '100%'}

Cuerpo = html.Div([dbc.Toast([html.Div([html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()),width=480, height=140),
                                        html.Br(),
    dcc.Slider(
    id='dias_pandemia',min=0,
    max=7,
    value=0,
    marks={
        0: {'label': '01/05/2020'},
        1: {'label': '01/08/2020'},
        2: {'label': '01/11/2020'},
        3: {'label': '10/01/2021'},
        4: {'label': '10/03/2021'},
        5: {'label': '01/05/2021'},
        6: {'label': '20/06/2021'},
        7:{'label': '01/09/2021'}

    },
    included=False),html.Div(id='Resultados_mapa'),visdcc.Run_js(id='Correr_java')])],
                    header= 'Covid-19 contagion route risk stratification for some of the most concurred Transmilenio in peak hour 6:30 am. Check and un-check the boxes that correspond to the route you wish to explore. The risk score is calculated considering active cases and RT score within each city block'
                             ,header_style = Estilo_arriba_caja,
                   style=Estilo_caja)],style={
            'marginTop': 30,
            'marginBottom': 20,
            'marginLeft': 250,
            'marginRight': 250,
            'padding': 0,'background': "#f5f5f5","opacity": 1,})



######### ---------------- Esto es un callback -------- ####### 

@app.callback(
    [Output('Resultados_mapa', 'children'),
     Output('Correr_java', 'run')],
    [Input('dias_pandemia', 'value')])

def Decision_analisis(dias_pandemia): 
    java = ''' alert("''' + 'Nueva estimación de riesgo en rutas '+'calculada'+'''"); '''
    mapa = html.Section(
        children=[html.Iframe(id='Mapa_general', srcDoc=render_maps[dias_pandemia], 
                              height = 950, 
            style={'display': 'flex','width': '100%','border': 0,
               'top': 0,
               'left' : 0,
               'bottom': 0,
               'right': 0} )],
        style={
            'padding': 0,
            'margin': 0,'height': 'calc(100vh - 60px)', 'overflowY': 'scroll',
            'borderRadius': 0,
            'border': 'thin lightgrey ridge', 'border-color': '#c7bfbf', 'border-width': '0px',
            'background': '#f5f5f5',"opacity": 0.9
        })
    
    return(mapa,java)
    
########## ------------- Abajo ------------- ############# 
Abajo = html.Div([html.H6(
        children='Todos los derechos - Copyright ©  2021.',
        style={'textAlign': 'center','color': 'white'})])


app.layout = html.Div(children=[Arriba, dcc.Loading(id="loading-1", type="circle", fullscreen=False, 
                                                                  children=[Cuerpo]), Abajo])



############# --------------- Server ----------- ############# 
#serve(app.server,host='127.0.0.1',port=8050) #this connects with the server

######## ------- Desplegar servidor -------- ########
#server = create_server(app,threaded=True)
#server.run()
app.run_server(debug=False)

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [13/Nov/2021 15:47:50] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [13/Nov/2021 15:47:51] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [13/Nov/2021 15:47:51] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [13/Nov/2021 15:47:51] "GET /_favicon.ico?v=2.0.0 HTTP/1.1" 200 -
127.0.0.1 - - [13/Nov/2021 15:47:51] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [13/Nov/2021 15:47:51] "GET /_dash-component-suites/dash/dcc/async-slider.js HTTP/1.1" 304 -
127.0.0.1 - - [13/Nov/2021 15:48:04] "POST /_dash-update-component HTTP/1.1" 200 -
